### DMMML assignment 2

Submitted by:-

Ashray Anand(MDS201905)

Tanmey Rawal(MDS201938)


The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed.

The classification goal is to predict if the client will subscribe (yes/no) a term deposit (variable y).



First let's load the dataset

In [1]:
import pandas as pd
dataset=pd.read_csv("T:/sem2/DMML_DS/assignment/bank-data/bank-additional-full.csv",sep=";")

Let's take look at first 5 instance of data

In [ ]:
dataset.head()

In [ ]:
dataset.info()

So the good thing is we don't have any missing data, as we have 41188 entries and for each attribute we have 41188 non null values

For now we are considering unknown as class label, we will see it what do with it later

The Attribute Information tells us to not include duration attribute as this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

We are removing it for now

In [2]:
dataset=dataset.drop(["duration"],axis=1)

Let's do some EDA

In [ ]:
dataset.describe()

Bank targeted population is slightly old people

As 999pdays means the person was never contacted before, most of the persons are being contacted for first time


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
dataset.hist(bins=50, figsize=(20,15))
plt.show()

Let's look at some categorical variables

In [ ]:
dataset["job"].value_counts()

admin and blue collar are bank's main target customer

In [ ]:
dataset["marital"].value_counts()

Married are bank's main target customer



In [ ]:
dataset["education"].value_counts()

People with university degree are high school are bank's main target customer

In [ ]:
dataset["housing"].value_counts()

There is almost equal spread between people having housing loan or not

In [ ]:
dataset["loan"].value_counts()

People with personal loan seems to be bank's main target customer

In [ ]:
dataset["contact"].value_counts()

In [ ]:
dataset["month"].value_counts()

In [ ]:
dataset["day_of_week"].value_counts()

In [ ]:
dataset["poutcome"].value_counts() 

In [3]:
def function (instance):
    if(instance['pdays']==999):
        return 0;
    return 1;
dataset['pdays2']=dataset.apply(lambda instance: function(instance),axis=1)
#changing the value 999 in pdays column to  value 30 
def function1 (instance):
    if(instance['pdays']==999):
        return 30;
    return instance['pdays'];
dataset['pdays']=dataset.apply(lambda instance: function1(instance),axis=1)

### Creation of test Data set

Let's create test set and keep it aside for testing purpose

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test=train_test_split(dataset,test_size=0.2,random_state=2)

In [5]:
len(X_train),len(X_test)

(32950, 8238)

Separation of target values

In [6]:
y_train=X_train["y"].copy()

In [7]:
X_train=X_train.drop("y",axis=1)

In [8]:
y_test=X_test["y"].copy()

In [9]:
X_test=X_test.drop("y",axis=1)

Converting yes/no label to numerical labels(0,1)

In [10]:
from sklearn.preprocessing import LabelBinarizer
lb=LabelBinarizer()
y_train_binary=lb.fit_transform(y_train)
y_train_binary

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [ ]:
lb.classes_

0 is dummy variable for no and 1 is dummy variable for yes

In [11]:
y_test_binary=lb.transform(y_test)

In [12]:
num_attributes=["age","campaign","pdays","pdays2","previous","emp.var.rate","cons.price.idx","cons.conf.idx","euribor3m","nr.employed"]

In [13]:
cat_attributes=["job","marital","education","default","housing","loan","contact","month","day_of_week","poutcome"]

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
num_pipeline = Pipeline([('std_scaler', StandardScaler()),])

In [15]:
from sklearn.preprocessing import OneHotEncoder
cat_pipeline=Pipeline([("one_hot_enc",OneHotEncoder()),])

In [16]:
from sklearn.compose import ColumnTransformer
full_pipeline=ColumnTransformer([
    ("num",num_pipeline,num_attributes),
    ("cat",cat_pipeline,cat_attributes)
])

In [17]:
X_transformed=full_pipeline.fit_transform(X_train)

In [18]:
from imblearn.under_sampling import NearMiss
nr = NearMiss()
X_transformed, y_train_binary = nr.fit_sample(X_transformed, y_train_binary)

## SVM

In [27]:
from sklearn.svm import SVC
svm_clf = SVC(gamma=0.0001,C=10)
svm_clf.fit(X_transformed, y_train_binary.ravel())

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [0.1], 'C': [1]},
                    {'kernel': ['linear'], 'C': [1]}]

In [ ]:

from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(SVC(), tuned_parameters, cv=5, scoring='precision')
clf.fit( X_transformed, y_train_binary.ravel())

In [28]:
from sklearn.model_selection import cross_val_score
cross_val_score(svm_clf, X_transformed, y_train_binary.ravel(), cv=3, scoring="accuracy",verbose=2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] ................................................. , total=   1.7s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV] ................................................. , total=   1.8s
[CV]  ................................................................
[CV] ................................................. , total=   1.8s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    5.3s finished


array([0.75887097, 0.79032258, 0.76532258])

In [ ]:
clf.best_params_

Now Predicting values using our SVM model

In [29]:
X_test_transformed=full_pipeline.transform(X_test)

In [30]:
y_pred_svm=svm_clf.predict(X_test_transformed)

In [31]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test_binary.ravel(),y_pred_svm)

array([[5493, 1825],
       [ 344,  576]], dtype=int64)

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test_binary,y_pred_svm)

0.736707938820102

In [33]:
from sklearn.metrics import  recall_score,precision_score,f1_score
recall_score(y_test_binary,y_pred_svm)

0.6260869565217392

In [34]:
f1_score(y_test_binary,y_pred_svm)

0.34688346883468835

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier()
tree_clf.fit( X_transformed, y_train_binary.ravel())

In [ ]:
cross_val_score(tree_clf, X_transformed, y_train_binary.ravel(), cv=10, scoring="accuracy")

In [ ]:
y_pred_tree=tree_clf.predict(X_test_transformed)

In [ ]:
confusion_matrix(y_test_binary.ravel(),y_pred_tree)

In [ ]:
accuracy_score(y_test_binary.ravel(),y_pred_tree)

In [ ]:
recall_score(y_test_binary,y_pred_tree)

In [ ]:
f1_score(y_test_binary,y_pred_tree)

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb_clf = GaussianNB()
nb_clf.fit(X_transformed, y_train_binary.ravel())

In [ ]:
cross_val_score(nb_clf, X_transformed, y_train_binary.ravel(), cv=10, scoring="accuracy")

In [ ]:
y_pred_nb=nb_clf.predict(X_test_transformed)

In [ ]:
confusion_matrix(y_test_binary.ravel(),y_pred_nb)

In [ ]:
accuracy_score(y_test_binary.ravel(),y_pred_nb)

In [ ]:
recall_score(y_test_binary,y_pred_nb)